In [1]:
### Imports for the Data Preprocessing
import numpy as np
import pandas as pd
import matplotlib as plt
import os
import gc
from typing import Tuple
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix




In [2]:
# get working directory
wd = os.path.dirname(os.getcwd())
# wd = "N:\MASTER_DS\Code\Kaggle_competition\Kaggle-seminar\student-performance"
os.chdir(wd)
print("Working Directory: ", os.getcwd())

Working Directory:  c:\Users\Stephan\Documents\Python\Python-Stat\Neuer\Kaggle_semiinar\student-performance


In [3]:
from utils.loader_steve import load_data, load_labels , load_all_X_y

df, labels = load_all_X_y( data= "flattened", )
keys = list(df.keys())

dtypes_labels= {
    'correct': np.uint8, 
    'q':np.uint8}

lab_1 = pd.read_csv('data/processed/labels_q1-3.csv', dtype=dtypes_labels)
lab_2 = pd.read_csv('data/processed/labels_q4-13.csv', dtype=dtypes_labels)
lab_3= pd.read_csv('data/processed/labels_q14-18.csv', dtype=dtypes_labels)

labels_dict = { "0_4": lab_1,
        "5_12": lab_2,
        "13_22": lab_3
}

In [4]:
keys = list(df.keys())
labels_dict = {
    "0_4": lab_1,
    "5_12": lab_2,
    "13_22": lab_3
}

# Create an empty dictionary to store the output DataFrames
output_dict = {}

# Iterate over each key in the original dictionary
for key in keys:
    train_data = df[key]
    train_data = train_data.sort_values('session_id_1')
    labels = labels_dict[key]
    labels.rename(columns={'session': 'session_id_1'}, inplace=True)
    labels = labels.sort_values('session_id_1')
    merged_df = pd.merge(train_data, labels, on="session_id_1", how="inner")
    merged_df = merged_df.drop(columns=[col for col in merged_df.columns if 'session' in col])
    grouped_df = merged_df.sort_values('q')
    
    # Group the merged DataFrame by the "q" column
    grouped_by_level = grouped_df.groupby('q')
    
    # Iterate over each unique level in the grouped DataFrame
    for level, group_data in grouped_by_level:
        # Create a unique key for the output dictionary
        output_key = f"{level}"
        # Drop the "level_group" and "q" columns from the group_data DataFrame
        group_data = group_data.drop(columns=["level_group_1", "q"])
        # Store the group_data DataFrame in the output dictionary
        output_dict[output_key] = group_data



In [24]:
check_df = output_dict["18"]

In [ ]:
#better xgboost model

In [5]:
'''import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import pandas as pd

# Dictionary to store the models and evaluation metrics
results = {}

# Hyperparameters to be tuned
param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300],
    'gamma': [0, 1, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'scale_pos_weight': [1, 5, 10]
}

# Iterate over each group of questions
for group_key, group_df in output_dict.items():
    # Group the DataFrame by the "level" column
    grouped_df = group_df.groupby('q')

    # Prepare lists to store the grouped data
    X_group = []
    y_group = []

    # Iterate over each group within the current group of questions
    for level, group_data in grouped_df:
        # Remove unnecessary columns
        group_data = group_data.drop(['q', 'correct', 'level_group_1'], axis=1)

        # Append the features and labels to the grouped data lists
        X_group.append(group_data)
        y_group.append(output_dict[group_key].loc[group_data.index]['correct'])

    # Concatenate the grouped data into single DataFrames
    X_group = pd.concat(X_group)
    y_group = pd.concat(y_group)

    # Perform basic cross-validation for hyperparameter tuning
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Create an XGBoost classifier
    model = xgb.XGBClassifier(random_state=42)

    # Perform grid search to find the best hyperparameters
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=skf, scoring='f1_macro')
    grid_search.fit(X_group, y_group)

    # Retrieve the best hyperparameters and their corresponding evaluation metrics
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    
    # Train the model on the entire dataset using the best hyperparameters
    best_model = xgb.XGBClassifier(**best_params, random_state=42)
    best_model.fit(X_group, y_group)

    # Store the best model and evaluation metrics for the current group of questions
    results[group_key] = {
        'Best Params': best_params,
        'Best F1 Score': best_score,
        'Model': best_model
    }



# Print the evaluation metrics and best hyperparameters
    for group_key, metrics in results.items():
        print('Group:', group_key)
        print('Best Params:', metrics['Best Params'])
        print('Best F1 Score:', metrics['Best F1 Score'])
        print('\n')


    # Make predictions on the entire dataset
        y_pred = best_model.predict(X_group)

        # Calculate evaluation metrics on the entire dataset
        f1 = f1_score(y_group, y_pred)
        precision = precision_score(y_group, y_pred)
        recall = recall_score(y_group, y_pred)
        conf_matrix = confusion_matrix(y_group, y_pred)

        # Store the evaluation metrics in the results dictionary
        metrics['F1 Score'] = f1
        metrics['Precision'] = precision
        metrics['Recall'] = recall
        metrics['Confusion Matrix'] = conf_matrix'''

'import xgboost as xgb\nfrom sklearn.model_selection import StratifiedKFold, GridSearchCV\nfrom sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix\nimport pandas as pd\n\n# Dictionary to store the models and evaluation metrics\nresults = {}\n\n# Hyperparameters to be tuned\nparam_grid = {\n    \'max_depth\': [3, 6, 9],\n    \'learning_rate\': [0.1, 0.01, 0.001],\n    \'n_estimators\': [100, 200, 300],\n    \'gamma\': [0, 1, 5],\n    \'subsample\': [0.8, 1.0],\n    \'colsample_bytree\': [0.8, 1.0],\n    \'scale_pos_weight\': [1, 5, 10]\n}\n\n# Iterate over each group of questions\nfor group_key, group_df in output_dict.items():\n    # Group the DataFrame by the "level" column\n    grouped_df = group_df.groupby(\'q\')\n\n    # Prepare lists to store the grouped data\n    X_group = []\n    y_group = []\n\n    # Iterate over each group within the current group of questions\n    for level, group_data in grouped_df:\n        # Remove unnecessary columns\n       

In [8]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# Dictionary to store results
results_dict = {}
test_data_dict = {}

# Iterate over each key (model) in the output_dict
for key, data in output_dict.items():
    X = data.drop("correct", axis=1)  # Features
    y = data["correct"]  # Labels

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    test_data_dict[key] = {
        "X_test": X_test,
        "y_test": y_test
    }

    # Define hyperparameter values to search for each model
    param_grid = {
        'max_depth': [3, 6, 9],
        'learning_rate': [0.1, 0.01, 0.001],
        'n_estimators': [100, 200, 300],
        'gamma': [0, 1, 5],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'scale_pos_weight': [1, 5, 10]
    }

    # Create XGBoost classifier
    model = xgb.XGBClassifier()

    # Perform grid search with cross-validation for the current model
    grid_search = GridSearchCV(model, param_grid, cv=10, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Get the grid search results for each hyperparameter combination
    cv_results = grid_search.cv_results_

    # Store results for the current model
    results_dict[key] = {
        "cv_results": cv_results,
        "best_params": grid_search.best_params_,
        "best_score": grid_search.best_score_
    }


KeyboardInterrupt: 

In [ ]:
# gpu training 

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# Enable GPU training
param = {'tree_method': 'gpu_hist'}

# Dictionary to store results
results_dict = {}
test_data_dict = {}

# Iterate over each key (model) in the output_dict
for key, data in output_dict.items():
    X = data.drop("correct", axis=1)  # Features
    y = data["correct"]  # Labels

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    test_data_dict[key] = {
        "X_test": X_test,
        "y_test": y_test
    }

    # Define hyperparameter values to search for each model
    param_grid = {
        'max_depth': [3, 6, 9],
        'learning_rate': [0.1, 0.01, 0.001],
        'n_estimators': [100, 200, 300],
        'gamma': [0, 1, 5],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'scale_pos_weight': [1, 5, 10]
    }

    # Create XGBoost classifier
    model = xgb.XGBClassifier(**param)

    # Perform grid search with cross-validation for the current model
    grid_search = GridSearchCV(model, param_grid, cv=10, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Get the grid search results for each hyperparameter combination
    cv_results = grid_search.cv_results_

    # Store results for the current model
    results_dict[key] = {
        "cv_results": cv_results,
        "best_params": grid_search.best_params_,
        "best_score": grid_search.best_score_
    }
